In [66]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_radio_stations_in_New_York'

In [28]:
#Read In HTML
tables = pd.read_html(url)
#Create a Dateframe From First Table
table_df = pd.DataFrame(tables[0])
#Set Column Names From Index 0
table_df.columns = table_df.iloc[0]
#Drop Duplicate Row[0]
table_df = table_df.reindex(table_df.index.drop(0))
#Rename Weird Columns
table_df = table_df.rename(columns={"City of License [1][2]":"City of License","Format [3]":"Format"})
#Sample Date
table_df

#table_df.groupby('Format')['City of License'].count()

,Call sign,Frequency,City of License,Licensee,Format
1,WAAL,99.1 FM,Binghamton,"Townsquare Media Binghamton License, LLC",Classic rock
2,WABC,770 AM,New York City,Radio License Holdings LLC,News/Talk
3,WABH,1380 AM,Bath,"Tower Broadcasting, LLC",Sports
4,WABY,900 AM,Saratoga Springs,Empire Broadcasting Corporation,Silent
5,WACK,1420 AM,Newark,"Waynco Radio, Inc.",News/Talk Information
6,WADO,1280 AM,New York City,"Univision Radio Illinois, Inc.",Spanish News and Talk
7,WAER,88.3 FM,Syracuse,Syracuse University,Public radio
8,WAIB-LP,88.5 FM,Redwood,"Betterarts, Inc.",Variety
9,WAIH,90.3 FM,Potsdam,State University of New York,Mixed
10,WAIO,95.1 FM,Honeoye Falls,"Citicasters Licenses, Inc.",Classic rock


In [4]:
county_url = "https://en.wikipedia.org/wiki/List_of_towns_in_New_York_(state)"



In [5]:
county_tables = pd.read_html(county_url)

In [57]:
county_df = pd.DataFrame(county_tables[1])

In [7]:
#Set Column Names From Index 0
county_df.columns = county_df.iloc[0]
#Drop Duplicate Row[0]
county_df = county_df.reindex(county_df.index.drop(0))
county_df.head()

,Town,County,Pop.(2010),Land(sq mi),Water(sq mi),Coordinates,GEO ID(FIPS code),ANSI code(GNIS ID)
1,Adams,Jefferson,5143,42.270,0.153,"43.844658, -76.054289",3604500210,00978655
2,Addison,Steuben,2595,25.545,0.142,"42.132164, -77.234422",3610100287,00978656
3,Afton,Chenango,2851,45.836,0.679,"42.226305, -75.526838",3601700353,00978657
4,Alabama,Genesee,1869,42.370,0.406,"43.090053, -78.366480",3603700474,00978658
5,Albion,Orleans,8468,25.257,0.093,"43.226343, -78.209387",3607301044,00978660


In [8]:
#Select Useful Columns
county_df = county_df[['Town','County']]


In [9]:
#Merge two data-frames
final_scrape_df = pd.merge(county_df, table_df, on='Town')

In [72]:
final_scrape_df.columns=final_scrape_df.columns.str.lstrip()
final_scrape_df.columns=final_scrape_df.columns.str.rstrip()
final_scrape_df.head()
#final_scrape_df.to_csv("radio.csv")

In [11]:
#Ping the endpoint thing
population_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H010001&for=county:*&in=state:36&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"


In [44]:
# Request data
population = requests.get(population_url).json()
pprint(population)


[['NAME', 'H010001', 'state', 'county'],
 ['Genesee County, New York', '58171', '36', '037'],
 ['Greene County, New York', '45821', '36', '039'],
 ['Livingston County, New York', '59635', '36', '051'],
 ['Herkimer County, New York', '63093', '36', '043'],
 ['Hamilton County, New York', '4756', '36', '041'],
 ['Jefferson County, New York', '110059', '36', '045'],
 ['Kings County, New York', '2469091', '36', '047'],
 ['Lewis County, New York', '26774', '36', '049'],
 ['Niagara County, New York', '212150', '36', '063'],
 ['Madison County, New York', '68309', '36', '053'],
 ['Monroe County, New York', '718061', '36', '055'],
 ['Montgomery County, New York', '49299', '36', '057'],
 ['Nassau County, New York', '1317866', '36', '059'],
 ['New York County, New York', '1518500', '36', '061'],
 ['Putnam County, New York', '97118', '36', '079'],
 ['Oneida County, New York', '221473', '36', '065'],
 ['Onondaga County, New York', '449957', '36', '067'],
 ['Ontario County, New York', '104602', '36',

In [84]:
#Make population a dataframe 
Pop_df = pd.DataFrame(population, columns= population[0])
Pop_df = Pop_df.drop([0])

#clean drop state and county columns 
Pop_df = Pop_df.drop(["state", "county"], axis=1)


#clean population table's name column by dropping "County, New York"
County = Pop_df.NAME.str.replace("County, New York", "")
County_df = pd.DataFrame(County)
County_df.columns=["County"]

Pop_df= pd.concat([Pop_df, County_df], axis=1)

#Rename population headers to pop_county, and total_pop, drop NAME, re-order
Pop_df = Pop_df.drop(["NAME"], axis=1)
Pop_df = Pop_df.rename(columns={"H010001": "Total_pop"})
Pop_df = Pop_df[['County', 'Total_pop']]
Pop_df['County']= Pop_df['County'].str.strip()
#County_df


Pop_df.to_csv("countypop.csv")
Pop_df.head()


,County,Total_pop
1,Genesee,58171
2,Greene,45821
3,Livingston,59635
4,Herkimer,63093
5,Hamilton,4756


In [94]:
#Merge county graph with population with final_scrape which has radio data 
pop_radio_df = pd.merge(Pop_df, final_scrape_df, on ='County')
pop_radio_df.head()

#Clean up formats
pop_radio_df["Format"] = pop_radio_df["Format"].replace(
    {'Adult album alternative/Talk':'Adult album alternative', 
     'Adult contemporary & News': 'Adult contemporary',
     'Alternative': 'Alternative radio',
     'Catholic Talk/EWTN' :'Catholic',
     'Christian (Family Life Network)':'Christian',
     'Christian Contemporary (Air1)': 'Christian',
     'Christian Contemporary (K-Love)': 'Christian',
     'Ethnic/Greek':'Ethnic',
    })
pop_radio_df


,County,Total_pop,Town,Call sign,Frequency,Licensee,Format
0,Genesee,58171,Batavia,WBTA,1490 AM,"HPL Communications, Inc.",Soft adult contemporary
1,Genesee,58171,Batavia,WGCC-FM,90.7 FM,Genesee Community College,Album-oriented rock
2,Greene,45821,Catskill,WCTW,98.5 FM,"CC Licenses, LLC",Hot adult contemporary
3,Greene,45821,Windham,WRIP,97.9 FM,"RIP Radio, LLC",Adult contemporary
4,Livingston,59635,Avon,WYSL,1040 AM,"Radio Livingston, Ltd.",News/Talk
5,Livingston,59635,Geneseo,WGSU,89.3 FM,State University of New York,Alternative radio
6,Herkimer,63093,Frankfort,WKLL,94.9 FM,Galaxy Utica Licensee LLC,Active rock
7,Herkimer,63093,Herkimer,WNRS,1420 AM,Arjuna Broadcasting Corp.,Oldies
8,Herkimer,63093,Herkimer,WVHC,91.5 FM,Herkimer County Community College,Variety
9,Herkimer,63093,Herkimer,WXUR,92.7 FM,Arjuna Broadcasting Corporation,Album Rock


In [97]:
#Determine # of formats per county
format_ser= pop_radio_df.groupby(['County', 'Format']).Format.count()
format_df=format_ser.unstack()
format_df_clean= format_df.fillna(0)

format_df_clean.head()

#Add population

final_df = pd.merge(Pop_df, format_df_clean, on ='County')
final_df.head()

#cleanup since I'm getting duplicate column name error in sqlite
headers = list(final_df.columns.values)
headers

['County',
 'Total_pop',
 '80s Hits',
 'AAA, progressive, blues, R&B, jazz & folk',
 'AAA/NPR News',
 'AC/Talk',
 'Active rock',
 'Adult Standards',
 'Adult album alternative',
 'Adult contemporary',
 'Adult hits',
 'Adult standards',
 'Album Rock',
 'Album-oriented rock',
 'Alternative radio',
 'Alternative rock',
 'Americana',
 'Bollywood (Indian) Music',
 'Catholic',
 'Christian',
 'Classic country',
 'Classic hip-hop',
 'Classic hits',
 'Classic rock',
 'Classical',
 'College',
 'Contemporary Christian',
 'Country',
 'Country music',
 'Ethnic',
 'Freeform',
 'Full service',
 'Full service, "Free-Format", College',
 'Full-service/classic hits',
 'Hispanic',
 'Hot AC',
 'Hot adult contemporary',
 'Independent',
 'Jazz',
 'Jazz, Ethnic',
 'Jazz/Community',
 'Jazz/mixed',
 'Mainstream jazz',
 'Mainstream rock',
 'Mixed',
 'Modern rock',
 'NPR',
 'News/Talk',
 'News/Talk Information',
 'News/Talk/Public radio',
 'Oldies',
 'Other',
 'Public radio',
 'Regional Mexican',
 'Religious',
 'R

In [98]:
#push to SQL.

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


engine = create_engine('sqlite:////Users/kim.campbell/Documents/GitHub/NY State of Mind/ETL_NY_State_Of_Mind/ny_radio.sqlite')
Base.metadata.create_all(engine)

final_df.to_sql(
    name= "genre_by_county",
    con = engine,
    index = False,
    if_exists = 'replace'
)

OperationalError: (sqlite3.OperationalError) duplicate column name: Adult standards [SQL: '\nCREATE TABLE genre_by_county (\n\t"County" TEXT, \n\t"Total_pop" TEXT, \n\t"80s Hits" FLOAT, \n\t"AAA, progressive, blues, R&B, jazz & folk" FLOAT, \n\t"AAA/NPR News" FLOAT, \n\t"AC/Talk" FLOAT, \n\t"Active rock" FLOAT, \n\t"Adult Standards" FLOAT, \n\t"Adult album alternative" FLOAT, \n\t"Adult contemporary" FLOAT, \n\t"Adult hits" FLOAT, \n\t"Adult standards" FLOAT, \n\t"Album Rock" FLOAT, \n\t"Album-oriented rock" FLOAT, \n\t"Alternative radio" FLOAT, \n\t"Alternative rock" FLOAT, \n\t"Americana" FLOAT, \n\t"Bollywood (Indian) Music" FLOAT, \n\t"Catholic" FLOAT, \n\t"Christian" FLOAT, \n\t"Classic country" FLOAT, \n\t"Classic hip-hop" FLOAT, \n\t"Classic hits" FLOAT, \n\t"Classic rock" FLOAT, \n\t"Classical" FLOAT, \n\t"College" FLOAT, \n\t"Contemporary Christian" FLOAT, \n\t"Country" FLOAT, \n\t"Country music" FLOAT, \n\t"Ethnic" FLOAT, \n\t"Freeform" FLOAT, \n\t"Full service" FLOAT, \n\t"F